In [1]:
%load_ext watermark
%watermark -v -m  -u -n -p pandas,numpy,matplotlib,json,sklearn -a Filippo_Valle -g -r -b -w

Filippo_Valle 
last updated: Tue Jun 30 2020 

CPython 3.7.6
IPython 7.15.0

pandas 1.0.4
numpy 1.18.5
matplotlib 3.2.1
json 2.0.9
sklearn 0.0

compiler   : GCC 7.5.0
system     : Linux
release    : 4.19.76-linuxkit
machine    : x86_64
processor  : x86_64
CPU cores  : 2
interpreter: 64bit
Git hash   : 728c5fcb2ae65fb2d06e3b61b8b167d388b13f95
Git repo   : git@github.com:fvalle1/topics.git
Git branch : develop
watermark 2.0.2


In [ ]:
import requests as rq
import json
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.cm as cm
from TCGA_files import get_tcga_tissue
#from ensembleAPI import geneinfo, genesinfo
from sklearn import metrics
from hsbmpy import *
import os

## query many

In [ ]:
level = 1
algorithm = 'topsbm'
directory = "/home/jovyan/work/phd/datasets/paper/gtex10seed"
L=get_max_available_L(directory, algorithm)
df_clusters = pd.read_csv("%s/%s/%s_level_%d_clusters.csv"%(directory,algorithm,algorithm,L), header=[0])
df_clusters.head()

In [ ]:
#df_files = pd.read_csv("https://storage.googleapis.com/gtex_analysis_v8/annotations/GTEx_Analysis_v8_Annotations_SampleAttributesDS.txt", sep='\t')
#df_files.set_index('SAMPID', inplace=True)
#df_files.dropna(how='all', axis=0).to_csv("%s/files.dat"%directory, index=True, header=True)

In [ ]:
df_files = pd.read_csv("%s/files.dat"%directory, index_col=[0], header=[0]).dropna(axis=1, how='all').dropna(axis=0, how='all')
samples = pd.read_csv("%s/%s/%s_level_0_clusters.csv"%(directory,algorithm,algorithm), header=[0]).astype(str).values.ravel()
samples=samples[samples!="nan"]
df_files = df_files.reindex(index=samples).dropna(how="all", axis=0).fillna("unknown")
labels = df_files.columns
labels

In [ ]:
#labels=['Subtype_Selected_Lum', 'BRCA_Subtype_PAM50']
#labels = ['cancer.type', "Subtype_Selected"]
labels=['SMTS', "SMTSD"]
#labels=["Type", "Subtype"]
#labels = ["primary_site", "tissue_hd"]

In [ ]:
for normalise in [True, False]:
    for label in labels:
        for level in np.arange(L+1)[::-1]:
            if level==0:
                #pass
                continue
            print(normalise, label, level)
            try:
                cluster = get_cluster_given_l(level, directory,algorithm=algorithm)
                fraction_sites = get_fraction_sites(cluster,df_files=df_files,label=label, normalise=normalise)

                clustersinfo = get_clustersinfo(cluster,fraction_sites)
                plot_cluster_composition(fraction_sites,directory,level,label=label, normalise=normalise,algorithm=algorithm)
                make_heatmap(fraction_sites, directory, label, level, normalise=normalise,algorithm=algorithm)

                clustersinfo = get_clustersinfo(cluster,fraction_sites)            
                if not normalise:
                    plot_maximum(clustersinfo,cluster,label,level, directory,algorithm=algorithm)
                    plot_maximum_size(clustersinfo,label,level, directory,algorithm=algorithm)
                    plot_maximum_label(clustersinfo,label,level, directory,algorithm=algorithm)
                    plot_sizes(level,directory, algorithm=algorithm)
            except:
                print(*sys.exc_info())
            continue
            shuffle_files(df_files,label).to_csv("%s/files_shuffles.dat"%directory, index=True)
            fraction_sites_shuffle = get_fraction_sites(cluster, df_files=pd.read_csv("%s/files_shuffles.dat"%directory, index_col=[0]),label=label, normalise=normalise)
            clustersinfo_shuffle = get_clustersinfo(cluster, fraction_sites_shuffle)
            plot_cluster_composition(fraction_sites_shuffle,directory,level, label=label, shuffled=True, normalise=normalise, algorithm=algorithm)
            if not normalise:
                plot_maximum(clustersinfo,cluster,label,level,directory,clustersinfo_shuffle,algorithm=algorithm)
                plot_maximum_size(clustersinfo,label,level, directory,clustersinfo_shuffle,algorithm=algorithm)
                plot_maximum_label(clustersinfo,label,level, directory,clustersinfo_shuffle,algorithm=algorithm)
                plot_labels_size(clustersinfo,label,level, directory,clustersinfo_shuffle,algorithm=algorithm)

In [ ]:
##useful for R clustree plot
def reindex(x):
    i=0
    last=x[0]
    new = []
    for xi in x:
        if xi == last:
            pass
        else:
            i+=1
            last=xi
        new.append(i)
    return new

df_labels = pd.DataFrame()
shape = df_clusters.dropna().shape
files = pd.read_csv("%s/%s/%s_level_4_clusters.csv"%(directory,algorithm,algorithm), header=[0]).astype(str).values.ravel()
files = files[files!="nan"]

for level in np.arange(L+1)[::-1]:
    df_clusters = pd.read_csv("%s/%s/%s_level_%d_clusters.csv"%(directory,algorithm,algorithm,level), header=[0])
    print("level ", level, "with ", df_clusters.shape[1], " clusters")
    currentlevellabels = []
    for file in files:
        s = df_clusters[df_clusters.isin([file])].any(0)
        a = s.index[s]
        currentlevellabels.append(int(a[0][8:])-1)
    df_labels.insert(0,'l%d'%level,currentlevellabels)
    del currentlevellabels
filelabels = []
filesublabels = []
for file in files:
    try:
        filelabels.append(get_file(file, df_files)[labels[0]])
        filesublabels.append(get_file(file, df_files)[labels[1]])
    except:
        filelabels.append('unknown')
        filesublabels.append('unknown')
        print(*sys.exc_info())
df_labels.insert(0,'tissue', filelabels)
df_labels.insert(0,'subtissue', filesublabels)
df_labels.sort_values(by=['tissue','subtissue'], inplace=True)
df_labels.sort_values(by=["l%d"%l for l in np.arange(L+1)[::-1]], axis=0, inplace=True)
filelabels = df_labels['tissue']
filesublabels = df_labels['subtissue']
df_labels = df_labels.apply(reindex, axis=0)
df_labels['tissue']=filelabels
df_labels['subtissue']=filesublabels
df_labels.to_csv("%s/%s/topsbm_labels.csv"%(directory,algorithm), index=False, header=True)

In [ ]:
if "clustersizes.txt" in os.listdir(directory):
    with open("%s/clustersizes.txt"%directory, 'r') as f:
        xl=np.array(f.read().split()).astype(int)
else:
    try:
        xl = getclustersizesarray(directory, L)
        with open("%s/clustersizes.txt" % directory, 'w') as f:
            for x in xl:
                f.write("%d\n" % x)
    except:
        print(*sys.exc_info())
        print("cannot save clustersizes.txt")

if "topicsizes.txt" in os.listdir(directory):
    with open("%s/topicsizes.txt"%directory) as f:
        tl=np.array(f.read().split()).astype(int)
else:
    try:
        tl = gettopicsizesarray(directory, L)
        with open("%s/topicsizes.txt" % directory, 'w') as f:
            for x in tl:
                f.write("%d\n" % x)
    except:
        print("cannot save topicsizes.txt")

In [ ]:
scores = get_scores(directory, labels, verbose=False)
scores['hsbm'] = scores[labels[0]]
scores['shuffle'] = get_scores_shuffled(directory, df_files, label=labels[0], algorithm='topsbm')

In [ ]:
fig=plt.figure(figsize=(18,15))
ax = fig.subplots(1)
add_score_lines(ax,scores,labels, alpha=1)
ax.set_xscale('log')
ax.set_ylim(0,1)

plt.show()
fig.savefig("%s/metric_scores_hierstructure.pdf"%(directory))

In [ ]:
fig=plt.figure(figsize=(18,15))
ax = fig.subplots(1)
#add_score_lines(ax,scores,[labels[0]],xl, h=True, c=True, alpha=1)
h = np.array(scores['hsbm']['h'])
c = np.array(scores['hsbm']['c'])
ax.plot(scores['hsbm']['xl'],h, marker='o', ls='--', lw=10, ms=40, label='homogeneity')
ax.plot(scores['hsbm']['xl'],c, marker='o', ls='-.', lw=10,ms=40, label='completeness')
ax.plot(scores['hsbm']['xl'],scores['hsbm']['V'], marker='o', ms=40, ls='-', lw=10, label='score')
ax.set_xscale('log')
ax.set_ylim(0,1)
#ax.plot(xl, 2*h*c/(h+c), ls='-',c='g')
plt.xticks(fontsize=35)
plt.yticks(fontsize=35)
plt.legend(fontsize=35)

ax.tick_params(labelsize=35, width=8, length=20)
ax.tick_params(which="minor", labelsize=35, width=5, length=15)

plt.xlabel('number of clusters', fontsize=35)
plt.ylabel('measure', fontsize=35)
plt.show()
fig.savefig("%s/metric_scores_primarysite.pdf"%(directory))

In [ ]:
scores['shuffle'] = get_scores_shuffled(directory, df_files, label=label, algorithm='topsbm')

In [ ]:
label = labels[0]
scores = {}
scores['hsbm'] = get_scores(directory, labels, df_files=df_files, verbose=False)[label]
scores['hierarchical'] = get_scores(directory, labels, algorithm='hierarchical-log', verbose=False)[label]
scores['lda'] = get_scores(directory, labels, algorithm='lda', verbose=False)[label]
#scores['kmeans'] = get_scores(directory, labels, algorithm='kmeans', verbose=False)[label]
scores['tm']=get_scores(directory, labels, algorithm='tm', verbose=False)[label]
#scores['cc']=get_scores(directory, labels, algorithm='cc', verbose=False)[label]
scores['wgcna']=get_scores(directory, labels, algorithm='wgcna', verbose=False)[label]
scores['shuffle'] = get_scores_shuffled(directory, df_files, label=label, algorithm='topsbm')

In [ ]:
label="SMTS"
scores={}
#scores['hierarchical'] = get_scores(directory, labels, algorithm='hierarchical-log', verbose=False)[label]
#scores['lda-tpm'] = get_scores(directory, labels, algorithm='lda-tpm', verbose=False)[label]
scores['hsbm-log2'] = get_scores(directory, labels, algorithm='topsbm-log', verbose=False)[label]
scores['hsbm-log10'] = get_scores(directory, labels, algorithm='topsbm-log10', verbose=False)[label]
scores['hsbm'] = get_scores(directory, labels, algorithm='topsbm', df_files=df_files, verbose=False)[label]
scores['hsbm-sweep'] = get_scores(directory, labels, algorithm='topsbm-sweep', df_files=df_files, verbose=False)[label]
scores['shuffle'] = get_scores_shuffled(directory, df_files, label=label, algorithm='topsbm')

In [ ]:
scores["tissues"] = get_scores(directory, labels, algorithm='topsbm', verbose=False)[labels[0]]
scores["sub_tissues"] = get_scores(directory, labels, algorithm='topsbm', verbose=False)[labels[1]]
scores[labels[0]] = scores["tissues"]
scores[labels[1]] = scores["sub_tissues"]

In [ ]:
from hsbmpy import normalise_score
if "mixed" in scores.keys():
    scores.pop("mixed")
norm_scores = scores.copy()
normalise_score(norm_scores, base_algorithm="hsbm")

In [ ]:
analyses = {
    "all": ['hsbm', 'tm', 'hierarchical', 'lda', 'wgcna', 'shuffle'],
    "subtypes": ["Subtype_Selected", "BRCA_Subtype_PAM50",'shuffle'],
    "hsbm": ["hsbm", "hsbm-sweep", "hsbm-log2","hsbm-log10",'shuffle'],
    "subtissues": ["tissues", "sub_tissues",'shuffle']
}

In [ ]:
analysis = "subtypes"

fig=plt.figure(figsize=(20,15))
ax = fig.subplots(1)
add_score_lines(ax,scores, labels=analyses[analysis])
#ax.set_xscale('linear')
plt.xlim(1,2000)
plt.show()
fig.savefig("%s/metric_scores_%s.pdf"%(directory,analysis))

In [ ]:
plt.bar(range(0,4*len(scores["hsbm"]["xl"]),4),scores["hsbm"]["V"], width = 1)
plt.bar(range(1,1+4*len(scores["lda"]["xl"]),4),scores["lda"]["V"], width = 1)
plt.bar(range(2,2+4*len(scores["hierarchical"]["xl"]),4),scores["hierarchical"]["V"], width = 1)

# Clustering consensus

In [ ]:
clusters = {}
labels = {}
algs = ["topsbm", "tm", "lda", "wgcna", "hierarchical-log"]
for algorithm, l in zip(algs,[1,0, 2, 1, 2]):
    clusters[algorithm]=pd.read_csv("%s/%s/%s_level_%d_clusters.csv"%(directory,algorithm,algorithm,l), header=[0])
    print(algorithm, clusters[algorithm].shape[1])
    labels[algorithm] = []
    for sample in samples:
        labels[algorithm].append(clusters[algorithm].columns[(clusters[algorithm]==sample).any()].values[0])

In [ ]:
from sklearn.metrics import v_measure_score
for alg1, alg2 in zip(["topsbm", "topsbm", "topsbm", "topsbm", "tm", "tm", "tm", "lda", "lda", "wgcna"],["tm","lda", "wgcna", "hierarchical-log", "lda", "wgcna", "hierarchical-log", "wgcna", "hierarchical-log", "hierarchical-log"]):
    print(alg1, " & ", alg2.replace("-log",""), " & ", "{:.3f}".format(float(v_measure_score(labels[alg1], labels[alg2]))), "\\\\ \hline")

In [ ]:
df_algs = pd.DataFrame(columns = algs, index=algs)
for alg1, alg2 in zip(["topsbm", "topsbm", "topsbm", "topsbm", "tm", "tm", "tm", "lda", "lda", "wgcna"],["tm","lda", "wgcna", "hierarchical-log", "lda", "wgcna", "hierarchical-log", "wgcna", "hierarchical-log", "hierarchical-log"]):
    df_algs.at[alg1,alg2]=float(v_measure_score(labels[alg1], labels[alg2]))
    
df_algs.at["tm", "topsbm"]=0.256
df_algs.at["lda", "topsbm"]=0.051
df_algs.at["wgcna", "topsbm"]=0.058
df_algs.at["lda", "tm"]=0.264
df_algs.at["wgcna", "tm"]=0.656
df_algs.at["wgcna", "lda"]=0.175
import seaborn as sns

cm = sns.clustermap(df_algs.fillna(0.5), vmin=0, vmax=1, row_cluster=False, 
                    col_cluster=False,
                   cmap = sns.diverging_palette(240, 10, sep=20, as_cmap=True))
ax = cm.ax_heatmap
fig = ax.get_figure()
ax.set_yticklabels(labels=algs, rotation=0)
ax.yaxis.tick_left()
ax.yaxis.set_label_position("left")

ax.set_xticklabels(labels=algs, rotation=90)
ax.tick_params(labelsize=35)

ax.text(1,4,"topics", fontsize=35, color="black", rotation = 360-45)
ax.text(2.5,2,"clusters", fontsize=35, color="black", rotation = 360-45)


cax = cm.ax_cbar
cax.tick_params(labelsize=30)
cax.set_title("$NMI$", fontsize=30)
plt.tight_layout()
cm.savefig(f"{directory}/comparisons.pdf")

plt.show()